## On plot les courbes Cl

In [10]:
import numpy as np
import torch 
from model import PINNs
import pandas as pd
from utils import charge_data
import json
import matplotlib.pyplot as plt
from plot_results import  plot_loss_decompose, plot_points, plot_loss
from model import PINNs
from utils import charge_data
import json
import torch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
import numpy as np
import time as time
import pandas as pd
from constants import DICT_CASE, DICT_Y0
from geometry import RectangleWithoutCylinder

In [11]:
folder_name = "10_huge"
epoch = "epoch820"

In [12]:
param_adim = {"V": 1.0, "L": 0.025, "rho": 1.2}

In [13]:
with open("results/" + folder_name + "/hyper_param.json", "r") as file:
    hyper_param = json.load(file)
model = PINNs(hyper_param)
checkpoint = torch.load(
    "results/" + folder_name + "/" + epoch + "/" + "model_weights.pth",
    map_location=torch.device("cpu"),
)
model.load_state_dict(checkpoint["model_state_dict"])


<All keys matched successfully>

In [14]:
with open('results/'+folder_name+'/mean_std.json') as file: 
    mean_std = json.load(file)

In [15]:
def find_cl_data(num, case, time_init, time_max, param_adim, r=0.025/2):
    df = pd.read_csv('data/'+ f"data_john_{num}_case_{case}.csv")
    df_modified = df.loc[
            (df["Time"] > time_init)
            & (df["Time"] < time_init + 15 * (time_max- time_init))
            & (df["Points:2"] == 0.0)
            & (df["Points:0"] ** 2 + df["Points:1"] ** 2 < 1.1* (0.025 / 2) ** 2),
            :,
        ].copy()
    X_theta = np.empty((df_modified.shape[0], 3))
    X_theta[:, 0] = time_init + ((df_modified['Time'].to_numpy() - time_init) % (time_max- time_init) )    #X_full[masque_theta][:, 2] 
    X_theta[:, 2] = df_modified['Pressure'].to_numpy()    #U_full[masque_theta][:, 2]
    X_theta[:, 1] = np.atan2(df_modified['Points:1'].to_numpy(), df_modified['Points:0'].to_numpy())    #np.atan2(X_full[masque_theta][:, 1] * mean_std['y_std'] + mean_std['y_mean'], X_full[masque_theta][:, 0] * mean_std['x_std'] + mean_std['x_mean'])
    portance = np.empty(np.unique(X_theta[:, 0]).shape[0])
    for nb, time_ in enumerate(np.unique(X_theta[:, 0])):
        masque_time = X_theta[:, 0] == time_
        indices = np.argsort(X_theta[masque_time][:, 1])[::-1]
        d_theta = X_theta[masque_time][indices][:-1, 1] - X_theta[masque_time][indices][1:, 1]
        portance[nb] = -np.sum(X_theta[masque_time][indices][1:, 2] * np.sin(X_theta[masque_time][indices][1:, 1])*d_theta * r)
    time_predict = np.unique(X_theta[:, 0])    # (X_full[masque_theta][:, 2].unique()* mean_std['t_std'] + mean_std['t_mean']) * (param_adim['L']/ param_adim['V'])
    return time_predict, portance / (0.5 * param_adim['rho'] * param_adim['V']**2 * 2 * r)
    

In [16]:
def true_cl(num, case, time_init, time_max):
    """
    Args:
        num (int): Le numéro du modèle
        case (int): Le numéro du case
        time_init (float): le temps initial 
        time_max (float): le temps max


    Returns:
        time (np.array):  les valeurs de temps
        cl (np.array): la valeur de cl à ces temps
    """
    df = pd.read_csv(f'results_Cl/model_{num}_case_{case}/Cl.csv', index_col=False)
    df_modified = df[((df.iloc[:, 2] > time_init) & (df.iloc[:, 2] < time_max))]
    time = df_modified.iloc[:, 2].to_numpy()
    cl = df_modified.iloc[:, 1].to_numpy()
    return time, cl


On a pour la force de portance : 

$
\overrightarrow{F_p}=-\int_{\theta=0}^{2\pi}(P(\theta)sin(\theta)rd\theta) \overrightarrow{u_y}
$

In [17]:
def P(theta, t, ya0, w0, model, param_adim, mean_std, r = 0.025 / 2):
    """
    Args:
        theta (torch.tensor): les theta où on veut calculer
        t_ad (torch.tensor): Les temps où on veut la pression en theta
        ya0 (float) : la valeur d'amplitude pour estimer notre pression
        model (torch.model): le NN qui prédit 
        param_adim (dict): dictionnaire des param d'adim
        mean_std (dict): donne de quoi faire la normalisation Z
        r (float): le rayon du cylindre

    Returns:
        P (torch.tensor):  les valeurs de pression sur le cylindre
    """
    x = r * torch.cos(theta)
    y = r * torch.sin(theta)
    x_ad = ((x / param_adim['L']) - mean_std['x_mean'])/mean_std['x_std']
    y_ad = ((y / param_adim['L']) - mean_std['y_mean'])/mean_std['y_std']
    t_ad = ((t / (param_adim['L']/param_adim['V'])) - mean_std['t_mean']) / mean_std['t_std']
    ya0_ad = torch.ones_like(x_ad) * ((ya0 / param_adim['L']) - mean_std['ya0_mean']) / mean_std['ya0_std']
    w0_ad = torch.ones_like(x_ad) * ((w0 / (param_adim['V']/param_adim['L'])) - mean_std['w0_mean']) / mean_std['w0_std']
    X = torch.stack((x_ad, y_ad, t_ad, ya0_ad, w0_ad), dim=1)
    P_adim = model(X)[:, 2]
    P = ((P_adim * mean_std['p_std'] + mean_std['p_mean']))*((param_adim['V']**2)*param_adim['rho'])
    return P, x, y

In [18]:
def predict_cl(t, ya0, w0,  model, param_adim, mean_std, r=0.025 / 2, nb_disc_theta=500):
    """
    Args:
        t_ad (torch.tensor): Les temps où on veut Cl
        ya0 (float) : la valeur d'amplitude pour estimer notre Cl
        model (torch.model): le NN qui prédit 
        param_adim (dict): dictionnaire des param d'adim
        mean_std (dict): donne de quoi faire la normalisation Z
        r (float): le rayon du cylindre
        nb_disc_theta(int): sur combien de theta on discrétise

    Returns:
        Cl (torch.float):  les valeurs de pression sur le cylindre
    """
    time_repeat = torch.repeat_interleave(t, repeats=nb_disc_theta)
    d_theta = 2 * torch.pi / nb_disc_theta
    theta_int = torch.linspace(0, 2*torch.pi, nb_disc_theta)
    theta_tot = theta_int.repeat(t.shape[0])
    pressure_tot, x, y = P(theta_tot, time_repeat, ya0, w0, model, param_adim, mean_std, r = 0.025 / 2) # .reshape(t.shape[0], nb_disc_theta) # (t.shape, nb_disc_theta)
    pressure_tot = pressure_tot.reshape(t.shape[0], nb_disc_theta)
    in_sum = pressure_tot * torch.sin(theta_int) * r * d_theta
    portance = - torch.sum(in_sum, dim=1)
    return portance/ (0.5 * param_adim['rho'] * param_adim['V']**2 * 2 * r) , x, y

In [23]:
num = [
    4,
    7,
    10,
    13, 
    4,
    7,
    10,
    13
]

case = [
    1,
    1,
    1,
    1,
    2,
    2,
    2,
    2,
]

files = [f'model_{num[k]}_case_{case[k]}.csv' for k in range(len(num))]

ya0_L = [DICT_Y0[str(k)] for k in num]
H_L = [DICT_CASE[str(k)] for k in case]

In [28]:
ya0_L = hyper_param['ya0']
H_L=hyper_param['H']
num = [
    int(hyper_param['file'][0][6])
]

case = [
    int(hyper_param['file'][0][13]),
]

files = hyper_param['file']

In [24]:
f = np.min(0.5 * (np.array(H_L) / hyper_param["m"]) ** 0.5)

In [25]:

t_max = hyper_param['t_min'] + 1 / f 
t = torch.linspace(hyper_param['t_min'], t_max, 1000)



In [26]:
def find_cl_data(num, case, time_init, T, param_adim, r=0.025/2):
    df = pd.read_csv('data/'+ f"data_john_{num}_case_{case}.csv")
    df_modified = df.loc[
            (df["Time"] > time_init)
            & (df["Time"] < time_init + 15 * T)
            & (df["Points:2"] == 0.0)
            & (df["Points:0"] ** 2 + df["Points:1"] ** 2 < 1.1* (0.025 / 2) ** 2),
            :,
        ].copy()
    X_theta = np.empty((df_modified.shape[0], 3))
    X_theta[:, 0] = time_init + ((df_modified['Time'].to_numpy() - time_init) % T )    #X_full[masque_theta][:, 2] 
    X_theta[:, 2] = df_modified['Pressure'].to_numpy()    #U_full[masque_theta][:, 2]
    X_theta[:, 1] = np.atan2(df_modified['Points:1'].to_numpy(), df_modified['Points:0'].to_numpy())    #np.atan2(X_full[masque_theta][:, 1] * mean_std['y_std'] + mean_std['y_mean'], X_full[masque_theta][:, 0] * mean_std['x_std'] + mean_std['x_mean'])
    portance = np.empty(np.unique(X_theta[:, 0]).shape[0])
    for nb, time_ in enumerate(np.unique(X_theta[:, 0])):
        masque_time = X_theta[:, 0] == time_
        indices = np.argsort(X_theta[masque_time][:, 1])[::-1]
        d_theta = X_theta[masque_time][indices][:-1, 1] - X_theta[masque_time][indices][1:, 1]
        portance[nb] = -np.sum(X_theta[masque_time][indices][1:, 2] * np.sin(X_theta[masque_time][indices][1:, 1])*d_theta * r)
    time_predict = np.unique(X_theta[:, 0])    # (X_full[masque_theta][:, 2].unique()* mean_std['t_std'] + mean_std['t_mean']) * (param_adim['L']/ param_adim['V'])
    return time_predict, portance / (0.5 * param_adim['rho'] * param_adim['V']**2 * 2 * r)
    

In [27]:
def find_cl_data_with_model(num, case, time_init, T, param_adim, ya0_, H_, r=0.025/2):
    df = pd.read_csv('data/'+ f"data_john_{num}_case_{case}.csv")
    df_modified = df.loc[
            (df["Time"] > time_init)
            & (df["Time"] < time_init + 15 * T)
            & (df["Points:2"] == 0.0)
            & (df["Points:0"] ** 2 + df["Points:1"] ** 2 < 1.1* (0.025 / 2) ** 2),
            :,
        ].copy()
    X_theta = np.empty((df_modified.shape[0], 3))
    X_theta[:, 0] = time_init + ((df_modified['Time'].to_numpy() - time_init) % T )    #X_full[masque_theta][:, 2] 
    #X_theta[:, 2] = df_modified['Pressure'].to_numpy()    #U_full[masque_theta][:, 2]
    X_theta[:, 1] = np.atan2(df_modified['Points:1'].to_numpy(), df_modified['Points:0'].to_numpy())    #np.atan2(X_full[masque_theta][:, 1] * mean_std['y_std'] + mean_std['y_mean'], X_full[masque_theta][:, 0] * mean_std['x_std'] + mean_std['x_mean'])
    x = (((torch.cos(torch.tensor(X_theta[:, 1])) * 0.025/2)/param_adim['L'])- mean_std['x_mean'])/mean_std['x_std']
    y = (((torch.sin(torch.tensor(X_theta[:, 1])) * 0.025/2)/param_adim['L'])- mean_std['y_mean'])/mean_std['y_std']
    time = torch.from_numpy(((X_theta[:, 0] / (param_adim['L']/ param_adim['V'])) - mean_std['t_mean'])/mean_std['t_std'])
    ya0 = ((((ya0_ / param_adim['L'])) - mean_std['ya0_mean'])/mean_std['ya0_std'])*torch.ones_like(x)
    w0 = torch.pi *(H_ / hyper_param["m"]) ** 0.5
    w0 = ((((w0 / (param_adim['V']/param_adim['L']))) - mean_std['w0_mean'])/mean_std['w0_std'])*torch.ones_like(x)
    X_theta[:, 2] = ((model(torch.stack((x,y,time,ya0,w0), dim=1).to(torch.float32)).detach().numpy()[:, 2]) * mean_std['p_std'] + mean_std['p_mean'])*(((param_adim["V"] ** 2) * param_adim["rho"]))
    portance = np.empty(np.unique(X_theta[:, 0]).shape[0])
    for nb, time_ in enumerate(np.unique(X_theta[:, 0])):
        masque_time = X_theta[:, 0] == time_,
        indices = np.argsort(X_theta[masque_time][:, 1])[::-1]
        d_theta = X_theta[masque_time][indices][:-1, 1] - X_theta[masque_time][indices][1:, 1]
        portance[nb] = -np.sum(X_theta[masque_time][indices][1:, 2] * np.sin(X_theta[masque_time][indices][1:, 1])*d_theta * r)
    time_predict = np.unique(X_theta[:, 0])    # (X_full[masque_theta][:, 2].unique()* mean_std['t_std'] + mean_std['t_mean']) * (param_adim['L']/ param_adim['V'])
    return time_predict, portance / (0.5 * param_adim['rho'] * param_adim['V']**2 * 2 * r)
    

In [28]:
for k in range(len(case)):
    w0 = torch.pi * (H_L[k]/hyper_param['m'])**0.5
    T = 2 * torch.pi/w0
    print(f"num :{num[k]}, case: {case[k]}")
    print(f"ya0: {ya0_L[k]}, H: {H_L[k]}")
    cl_predict, x, y = predict_cl(t, ya0_L[k], w0, model, param_adim, mean_std, nb_disc_theta=500)
    time, cl_true = true_cl(num=num[k], case=case[k], time_init=hyper_param['t_min'], time_max=t_max)
    with torch.no_grad():
        plt.plot(t, cl_predict, color = 'red', label = 'Predict Cl', linewidth=0.8)
        plt.plot(time, cl_true, label = 'True Cl', color='blue', linewidth=0.8)
        #plt.plot(time_data, cl_data, label = 'Data Cl', color='green', linewidth=0.8)
        plt.title('plot de Cl', fontsize=16, fontweight='bold')  # Titre
        plt.xlabel('Temps (s)', fontsize=14)  # Label de l'axe x
        plt.ylabel('Cl', fontsize=14)  # Label de l'axe y
        plt.legend(loc='upper right', fontsize=12)  # Légende
        plt.grid(alpha=0.7, linestyle= '--')  # Grille avec transparence
        plt.xticks(fontsize=12)  # Taille des ticks de l'axe x
        plt.yticks(fontsize=12)  # Taille des ticks de l'axe y
        plt.tight_layout()  # Ajuste le layout pour éviter le chevauchement
        os.makedirs('results/' + folder_name + f'/H_{H_L[k]}/ya0_{ya0_L[k]:.2e}', exist_ok=True)
        plt.savefig('results/' + folder_name + f'/H_{H_L[k]}/ya0_{ya0_L[k]:.2e}/cl.png')
        plt.show()
    

num :4, case: 1
ya0: 0.003125, H: 261.39


FileNotFoundError: [Errno 2] No such file or directory: 'results_Cl/model_4_case_1/Cl.csv'